## Welcher Produzenten könnte in Zukunft besonders erfolgreiche Animes produzieren?

Berechne den Spearman-Korrelationskoeffizienten zwischen der Anzahl erfolgreicher Animes eines Produzenten und der Anzahl der Verbindungen zu wichtigen Produzenten oder beliebten Animes.

Spearman-Korrelation misst den monotonen Zusammenhang zwischen den Rängen der Studioerfolgszahlen und den Verbindungen zu Top-Animes oder anderen Produzenten.

In [18]:
import pandas as pd

# Dateien laden
producer_relations_path = 'data/Producer_Relations.csv'
producer_nodes_path = 'data/Producer_Nodes.csv'
anime_nodes_path = 'data/Anime_Nodes.csv'

producer_relations = pd.read_csv(producer_relations_path)
producer_nodes = pd.read_csv(producer_nodes_path)
anime_nodes = pd.read_csv(anime_nodes_path)


# Titel und Tabellen anzeigen
print("\n===== Producer Relations =====")
print(producer_relations.head(10).to_string(index=False))  # Zeige die ersten 10 Zeilen als Tabelle

print("\n===== Producer Nodes =====")
print(producer_nodes.head(10).to_string(index=False))  # Zeige die ersten 10 Zeilen als Tabelle

print("\n===== Anime Nodes =====")
print(anime_nodes.head(10).to_string(index=False))  # Zeige die ersten 10 Zeilen als Tabelle



===== Producer Relations =====
 source  target       type
  52991      17 undirected
  52991      53 undirected
  52991      62 undirected
  52991    1003 undirected
  52991    1143 undirected
  52991    1430 undirected
   5114      17 undirected
   5114      58 undirected
   5114     143 undirected
   5114    1499 undirected

===== Producer Nodes =====
  id    label                      name
2015 Producer     HoriPro International
1916 Producer        Kadokawa Animation
1850 Producer         Rex Entertainment
1784 Producer Crunchyroll SC Anime Fund
1619 Producer             Bungeishunjuu
1586 Producer                       CIC
1553 Producer          Shounen Gahousha
2511 Producer                 Blue Lynx
1355 Producer                  TV Aichi
 199 Producer                Studio Nue

===== Anime Nodes =====
   id label                               name  score  rank                  studio
52991 Anime                  Sousou no Frieren   9.32     1              [Madhouse]
 5114 Anim

Die Anzahl der Verbindungen (Degree Centrality) wurde erfolgreich berechnet und den Produzenten zugeordnet. Zum Beispiel hat „HoriPro International“ zwei Verbindungen.